# Data Parallelism

## Overview

Single-node **Data Parallelism** (PyTorch `nn.DataParallel`) enables model training across multiple GPUs **within a single machine**. Unlike **DistributedDataParallel** (`nn.DistributedDataParallel`), which uses multiple processes, `nn.DataParallel` operates with a single process that splits data across GPUs. For larger models and multi-node setups, DistributedDataParallel is preferred over DataParallel.

### Key Steps:

1. **Model Replication:** The model is copied to each available GPU.
2. **Data Splitting:** The input batch is divided among GPUs.
3. **Forward Pass:** Each GPU computes outputs independently.
4. **Gathering Outputs:** Results are collected on the main GPU.
5. **Backward Pass:** Each GPU computes gradients locally.
6. **Gradient Synchronization:** Gradients from all GPUs are summed and reduced to the main GPU.
7. **Model Update:** The optimizer updates the model on the main GPU, and updated parameters are broadcast to other GPUs.

---

## Communication Flow

1. **Scatter (Data Distribution)**  
   - Input data is split and distributed across GPUs.
   
2. **Parallel Computation (Forward & Backward Passes)**  
   - Each GPU computes forward outputs and backward gradients independently.

3. **Gather (Gradient Aggregation & Synchronization)**  
   - Gradients are collected on the main GPU and summed.

4. **Model Update**  
   - The main GPU updates model parameters, which are then copied to other GPUs.

---

## Pros and Cons

Limits of the DP
- GIL Limitation: Due to Python's Global Interpreter Lock (GIL), DataParallel cannot fully utilize multiple GPUs efficiently.
- Uneven Load Distribution: The primary GPU (rank 0) is more heavily utilized than others, causing an imbalance in computation.
- Synchronization Overhead: Each training iteration requires synchronizing model weights across GPUs, increasing latency.
- Single-node Restriction: DataParallel only works within a single machine, making it unsuitable for true distributed training.

| Aspect               | `nn.DataParallel` |
|----------------------|------------------|
| **Ease of Use**      | Simple, just wrap `nn.DataParallel(model)` |
| **Process Management** | Single process, easier debugging |
| **Scalability**      | Limited to a single machine |
| **Bottlenecks**      | Main GPU handles gradient gathering, causing potential slowdowns |
| **Multi-Node Support** | Not supported |

---

## PyTorch Implementation Example

```python
import torch
import torch.nn as nn
import torch.optim as optim

# Define a model
model = MyModel()

# Enable DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# Training loop
for data, target in dataloader:
    data, target = data.cuda(), target.cuda()

    # Forward pass (DataParallel splits data automatically)
    outputs = model(data)
    loss = criterion(outputs, target)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()  # Gradients are reduced to the main GPU
    optimizer.step() # Model parameters on main GPU are updated
```

The hugging face Trainer uses Data Parallel by default. In this notebook, we will write a PyTorch implementation without the hugging face Trainer. Let's first build torch dataloader from huggingface dataset. There is also `YelpReviewFull` in `torchtext.datasets`, but it's no longer maintained and doesn't work for higher version of python/pytorch.

## Load Data

In [26]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import Dataset, DataLoader, RandomSampler, SubsetRandomSampler
import torch

# Load dataset
dataset = load_dataset("yelp_review_full")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

# # Tokenization function
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")

# # Tokenize dataset
# tokenized_datasets = dataset.map(tokenize_function, batched=True)
# # Remove columns not needed for training
# tokenized_datasets = tokenized_datasets.remove_columns(["text"])


# Define a custom Dataset class
class YelpReviewDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        text = item['text']
        label = item['label']
        # Tokenize the text
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
        # Squeeze to remove unnecessary dimensions
        output = {key: val.squeeze(0) for key, val in encoding.items()}
        output["labels"] = label
        return output

# Create Dataset instances for train and test
train_dataset = YelpReviewDataset(dataset['train'], tokenizer)
test_dataset = YelpReviewDataset(dataset['test'], tokenizer)

# Determine the number of samples to select
num_train_samples = 100
num_test_samples = 50

# Generate random indices for training and testing datasets
train_indices = torch.randperm(len(train_dataset)).tolist()[:num_train_samples]
test_indices = torch.randperm(len(test_dataset)).tolist()[:num_test_samples]

# Create SubsetRandomSamplers
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

# Create DataLoaders
train_dataloader = DataLoader(
    train_dataset,
    sampler=train_sampler,
    batch_size=32,
)

test_dataloader = DataLoader(
    test_dataset,
    sampler=test_sampler,
    batch_size=32,
    # drop_last=True
)

Using the latest cached version of the dataset since yelp_review_full couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'yelp_review_full' at /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/0.0.0/c1f9ee939b7d05667af864ee1cb066393154bf85 (last modified on Sat Feb 22 16:51:02 2025).


In [27]:
# Example: Iterate through DataLoader
for batch in train_dataloader:
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    print(input_ids.shape, attention_mask.shape, labels.shape)
    break  # Only print the first batch

torch.Size([32, 128]) torch.Size([32, 128]) torch.Size([32])


### About Padding
In the current setup, the tokenizer is configured with padding="max_length" and truncation=True. This means that:
- Padding: Each sequence is padded to a fixed length specified by the max_length parameter. If max_length is not explicitly set, it defaults to the maximum length accepted by the model.
- Truncation: Sequences longer than max_length are truncated to fit this length.

Therefore, padding is applied to the specified max_length, not dynamically to the length of the longest sequence in each batch. If you prefer padding to the length of the longest sequence in each batch (dynamic padding), you can adjust the padding strategy:
- Dynamic Padding: Set padding=True when tokenizing. This pads sequences to the length of the longest sequence in the batch, which can be more efficient in terms of computation and memory usage.

There are also advanced padding techniques. When training large language models (LLMs) with variable-length sequences, efficiently handling padding is crucial to optimize computational resources and maintain model performance. In PyTorch, the `pack_padded_sequence` and `pad_packed_sequence` functions are designed to manage padded sequences effectively, particularly when using recurrent neural networks (RNNs) like LSTMs or GRUs.

- **`pack_padded_sequence`**: This function converts a padded batch of sequences into a `PackedSequence` object, which RNNs can process more efficiently. It requires the sequences to be sorted by length in descending order and the lengths of each sequence.

- **`pad_packed_sequence`**: After processing with an RNN, this function converts the `PackedSequence` back into a padded tensor, restoring the original sequence lengths.

### Padding in LLM Training

While `pack_padded_sequence` and `pad_packed_sequence` are beneficial for RNNs, they are not directly applicable to transformer-based models commonly used in LLMs. Transformers handle variable-length sequences through attention mechanisms and positional encodings, allowing them to process sequences without the need for packing and unpacking.

For transformer-based models, efficient padding strategies are essential to minimize computational overhead:

1. **Dynamic Padding**: Pad sequences to the length of the longest sequence in each batch rather than the longest sequence in the entire dataset. This approach reduces unnecessary padding and computational waste.

2. **Masking**: Use attention masks to inform the model about padded tokens, ensuring they do not contribute to the attention mechanism. This practice prevents the model from learning from padding tokens.

Here's how you can implement dynamic padding and masking in a PyTorch DataLoader for transformer-based models:

```python
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def collate_fn(batch):
    # Tokenize the batch of texts
    encodings = tokenizer([item['text'] for item in batch], padding=True, truncation=True, return_tensors='pt')
    # Create attention masks
    attention_masks = (encodings['attention_mask'] == 1).float()
    # Stack labels
    labels = torch.tensor([item['label'] for item in batch])
    return encodings, attention_masks, labels

# Create DataLoader with dynamic padding
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=collate_fn)
```


## Load Model and Train

In [28]:
import torch.nn as nn
from torch.optim import Adam
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5, torch_dtype="auto")
print(model.config)

if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

if torch.cuda.is_available():
    model = model.cuda()

optimizer = Adam(model.parameters(), lr=2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "google-bert/bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.49.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



In [32]:
import time

def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in test_dataloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / num_test_samples

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        start = time.time()
        for batch in train_dataloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)

            # DP Output.loss will be tensors so need to take the mean
            loss = output.loss.mean()

            loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}, time: {time.time() - start}")

In [33]:
train()

ep: 0, global_step: 0, loss: 1.5722355842590332
ep: 0, acc: 0.2199999988079071, time: 0.9605724811553955
ep: 1, acc: 0.2199999988079071, time: 0.8442580699920654
ep: 2, acc: 0.1599999964237213, time: 0.8297109603881836


## Predict

In [34]:
%%time

# single GPU
with torch.inference_mode():
    for batch in train_dataloader:
        if torch.cuda.is_available():
            batch = {k: v.cuda() for k, v in batch.items()}
        output = model.module(**batch)



CPU times: user 263 ms, sys: 23.9 ms, total: 287 ms
Wall time: 251 ms


In [35]:
%%time

# multi GPU. Data Parallel can help
with torch.inference_mode():
    for batch in train_dataloader:
        if torch.cuda.is_available():
            batch = {k: v.cuda() for k, v in batch.items()}
        output = model(**batch)


CPU times: user 488 ms, sys: 32.2 ms, total: 520 ms
Wall time: 397 ms


In [39]:
%%time

# multi GPU. only replicate model once

replicated_model = model.replicate(model.module, device_ids=[0, 1])
with torch.inference_mode():
    for batch in train_dataloader:
        if torch.cuda.is_available():
            batch = {k: v.cuda() for k, v in batch.items()}
        inputs, module_kwarg = model.scatter(inputs=None, kwargs=batch, device_ids=model.device_ids)
        output = model.parallel_apply(replicated_model, inputs, module_kwarg)
        output = model.gather(output, model.output_device)

CPU times: user 276 ms, sys: 47.9 ms, total: 324 ms
Wall time: 254 ms
